# Bilinear Transformer Tutorial

This is a very quick guide on how to use our scuffed tooling to do interpretability on bilinear transformers.

In [1]:
from language import Transformer
from nnsight import LanguageModel

import plotly.express as px
from einops import *
import torch

torch.set_grad_enabled(False)

In [2]:
model = Transformer.from_pretrained(d_model=256, n_layer=4)
model.summary()

,name,parameters,dimensions
0,total,5507328,
1,emb.tok,1048576,256 x 4096
2,head,1048576,4096 x 256
3,attn.qkv,196608,3 x 256 x 256
4,attn.out,65536,256 x 256
5,mlp.bilin,393216,2 x 768 x 256
6,mlp.out,196608,256 x 768


We can generate some text with our models.
Keep in mind that these models are very small and brittle, let's take it slightly off distribution.

In [10]:
model.generate("Hello, world!")

'hello, world! fuzzy was playing in the backyard, so he saw a hedge. he had never seen an owl before. " you should think it may be good to me, " he said to himself. " why? ". but just a little bit, a butterfly flew by. " i think it\'ll be fun to try, " he said. " but it\'s so foolish! " so he jumped off the hedge and flew over to it. " why don\'t you try to touch it? " he asked. but the butterfly just kept flying away. confused. he asked the hedge solved the mystery and hovered through the grass. " you can learn bugs if you want, " said the hedge. " you mommy won\'t know what that is - chirping around it. " " juicagging the hedge is luining here! ", said the hedge. from then on, the hedge couldn\'t be for him anymore. he wanted to be happy again. he thanked the hedge for being so cool and quiet, and hopped away to explore some more. from then on, he was always careful no matter the decision. he was full of knowledge that he could achieve any moment'

In [11]:
model.generate("Once upon a time, the frog and the lizard")

'once upon a time, the frog and the lizard were walking through the town. he noticed how big the roads were and the road was very filthy. they were all covered in dirt, so the frog asked, " what is all that do? " the lizard said, " it\'s noon! take that time and clean. " then the frog and the lizard went off on a grand adventure. as they hopped on the path, they discovered all sorts of lovely things. they saw chickens, hills and even a rabbit! they were so busy enjoying the sun and singing and playing. the frog and the lizard wandered further until they came to a clearing. the sun was just down and there was no more. they were so excited, they just had found it! they looked for a new one, but they couldn\'t find one. the frog and lizard stopped to look around and noticed one bird was sitting on a branch and its eye was much red, so it wasn\'t very tidy. the bird was still sleeping, and the frog and lizard watched until the bird finally woke up. they were the most wonderful thing they h

In [ ]:
# prompt = ""
# prompt = "the color of the sky was"
# prompt = "the lizard and the frog"
# prompt = "once upon a time, it was raining, the grass was"
# prompt = "jimmy and his friend were at the zoo, jimmy wanted to see the largest animal. The largest animal is "
# prompt = "billy and john went to the park. billy gave a hug to"
# prompt = "once upon a time the boys"
# prompt = "once upon a time, bob was playing with a cat. bob's"
prompt = "the boys near the car"

model.generate(prompt, 50, temperature=1, top_k=2)

## Integration with NNSight
We can also perform activation tracing and patching with NNSight. Our models are (kinda) compatible with their ``LanguageModel`` class.

In [3]:
lm = LanguageModel(model, tokenizer=model.tokenizer)

In [5]:
prompt = "One time, John and Mary were in the park. John wanted to play a game with"
with lm.trace(prompt):
    qk = lm.transformer.h[3].attn.rotary.output.save()

q, k = qk.value
attn = einsum(q, k, "... seq_q d_head, ... seq_k d_head -> ... seq_q seq_k")
pattern = torch.softmax(attn.tril(), dim=-1)

labels = model.make_labels(prompt)
px.imshow(pattern[0].cpu(), x=labels, y=labels, facet_col=0)